In [1]:
!pip install --upgrade pip
!pip -q install langchain langchain-community openai tiktoken chromadb python-dotenv pypdf sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
poetry 1.2.0 requires requests-toolbelt<0.10.0,>=0.9.1, but you have requests-toolbelt 1.0.0 which is incompatible.
datasets 2.16.1 requires dill<0.3.8,>=0.3.0, but you have dill 0.3.8 which is incompatible.


# Create a Vector Database with ChromaDB

Data -> Embedding Model -> Store

## Setting up LangChain


In [ ]:
# raw (text pdf) -> embeddding -> vector -> database

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

In [2]:
from typing import List, Optional, Union
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import CrossEncoder

/Users/khoa/py39/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Load multiple and process documents

In [20]:
import os
from langchain.document_loaders import TextLoader, PyPDFLoader

DATA_PATH = "verra/"

# Function to load both PDF and TXT files based on file extension with error handling
def load_file(file_path):
    try:
        if file_path.endswith('.txt'):
            return TextLoader(file_path).load()
        elif file_path.endswith('.pdf'):
            return PyPDFLoader(file_path).load()
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
    return []

# Recursive function to traverse directories and load files
def load_documents_from_directory(base_path):
    all_documents = []
    for root, _, files in os.walk(base_path):
        for file in files:
            file_path = os.path.join(root, file)
            documents = load_file(file_path)
            all_documents.extend(documents)
    return all_documents

documents = load_documents_from_directory(DATA_PATH)

 impossible to decode XFormObject /Fm0
 impossible to decode XFormObject /Fm1
 impossible to decode XFormObject /Fm2
 impossible to decode XFormObject /Fm3
invalid pdf header: b'<!DOC'
EOF marker not found


Error loading file ../data/verra/verra_news/pdf/doc_4771.pdf: Stream has ended unexpectedly


invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!doc'
EOF marker not found


Error loading file ../data/verra/verra_news/pdf/doc_5242.pdf: Stream has ended unexpectedly
Error loading file ../data/verra/verra_news/pdf/83-cap_1.pdf: Stream has ended unexpectedly


invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'\n\n \n '
EOF marker not found
EOF marker not found
EOF marker not found


...

invalid pdf header: b'<html'
EOF marker not found
EOF marker not found


Error loading file ../data/verra/verra_news/pdf/paris_agreement_english_.pdf: Stream has ended unexpectedly


invalid pdf header: b'<html'
EOF marker not found
EOF marker not found


Error loading file ../data/verra/verra_views/pdf/pub_07_financial_flows.pdf: Stream has ended unexpectedly


EOF marker seems truncated
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 47 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 

In [22]:
def split_text(
    documents,
    separators=["\n\n\n", "\n\n"],
    chunk_size=1000,
    chunk_overlap=200,
):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=separators,
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

texts = split_text(documents)
print(f"Length text : {len(texts)}")
# Length text : 44873

Length text : 11839


## create the DB

In [15]:
def load_embedding_model(
    model_name: str = "Snowflake/snowflake-arctic-embed-m-v1.5", device: str = "cpu"
) -> HuggingFaceBgeEmbeddings:
    model_kwargs = {"device": device}
    encode_kwargs = {
        "normalize_embeddings": True
    }  # set True to compute cosine similarity
    embedding_model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
    return embedding_model


def generate_embeddings(docs, embedding_model):
    db = FAISS.from_documents(documents=docs, embedding=embedding_model)
    return db

In [16]:
import pickle
with open("/Users/khoa/Downloads/batch_0.pkl", "rb") as r:
    documents = pickle.load(r)

In [17]:
%%time
embedding_model = load_embedding_model()
vectorstore = generate_embeddings(documents, embedding_model=embedding_model)
vectorstore.save_local("document_idx")

CPU times: user 1h 42min 56s, sys: 33min 17s, total: 2h 16min 14s
Wall time: 19min 50s


In [39]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7682f99b7670>, search_kwargs={'k': 10})

In [52]:
query = """
Neijiang  Xingming  Energy  Co.,  Ltd  has  commissioned  Det  Norske  Veritas  Certification  AS 
(DNV)  to  perform  a  validation  of  the  “Chuanwei  Group  24  MW  Waste  Gas  Based  Captive 
Power  Plant”  project  in  China  (hereafter  called  “the  project”). 
"""

In [35]:
def load_reranker_model(
    reranker_model_name: str = "BAAI/bge-reranker-large", device: str = "cuda"
) -> CrossEncoder:
    reranker_model = CrossEncoder(
        model_name=reranker_model_name, max_length=512, device=device
    )
    return reranker_model
    
def rerank_docs(reranker_model, query, retrieved_docs):
    query_and_docs = [(query, r.page_content) for r in retrieved_docs]
    scores = reranker_model.predict(query_and_docs)
    return sorted(list(zip(retrieved_docs, scores)), key=lambda x: x[1], reverse=True)

In [54]:
reranker_model = load_reranker_model()

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
retrieved_documents = retriever.get_relevant_documents(query)
reranked_docs = rerank_docs(reranker_model, query, retrieved_documents)

In [55]:
reranked_docs

[(Document(metadata={'source': '../data/verra/projects/VCS/12/registration/Chuanwei CDM Validation Report.pdf', 'page': 5}, page_content='D ET  N ORSKE  V ERITAS  \n \nV ALIDATION R EPORT  \n \nCDM Validation2007-0849, rev.2\n 6  \n2  INTRODUCTION \nNeijiang Xingming Energy Co., Ltd has commissioned Det Norske Veritas Certification AS \n(DNV) to perform a validation of the “Chuanwei Grou p 24 MW Waste Gas Based Captive \nPower Plant” project in China (hereafter called “th e project”). This report summaries the \nfindings of the validation of the project, performe d on the basis of UNFCCC criteria for CDM \nprojects, as well as criteria given to provide for consistent project operations, monitoring and \nreporting.  \n2.1 Objective \nThe purpose of a validation is to have an independe nt third party assess the project design. In \nparticular, the project’s baseline, monitoring plan , and the project’s compliance with relevant \nUNFCCC and host Party criteria are validated in ord er to c

In [57]:
# Now we can load the persisted database from disk, and use it as normal.
# Path to the saved FAISS index
index_path = "faiss_index"

# Reload the FAISS index
vectordb = FAISS.load_local(index_path, embedding=embedding_model)

TypeError: FAISS.load_local() missing 1 required positional argument: 'embeddings'

In [58]:
help(FAISS)

Help on class FAISS in module langchain_community.vectorstores.faiss:

class FAISS(langchain_core.vectorstores.base.VectorStore)
 |  FAISS(embedding_function: 'Union[Callable[[str], List[float]], Embeddings]', index: 'Any', docstore: 'Docstore', index_to_docstore_id: 'Dict[int, str]', relevance_score_fn: 'Optional[Callable[[float], float]]' = None, normalize_L2: 'bool' = False, distance_strategy: 'DistanceStrategy' = <DistanceStrategy.EUCLIDEAN_DISTANCE: 'EUCLIDEAN_DISTANCE'>)
 |  
 |  FAISS vector store integration.
 |  
 |  See [The FAISS Library](https://arxiv.org/pdf/2401.08281) paper.
 |  
 |  Setup:
 |      Install ``langchain_community`` and ``faiss-cpu`` python packages.
 |  
 |      .. code-block:: bash
 |  
 |          pip install -qU langchain_community faiss-cpu
 |  
 |  Key init args — indexing params:
 |      embedding_function: Embeddings
 |          Embedding function to use.
 |  
 |  Key init args — client params:
 |      index: Any
 |          FAISS index to use.
 |  

In [1]:
import boto3
s3 = boto3.client('s3')

In [18]:
import boto3
import tempfile
from langchain_community.document_loaders import TextLoader, PyPDFLoader


def load_file_from_s3(key, bucket_name, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Download the file
            file_obj = s3.get_object(Bucket=bucket_name, Key=key)
            file_content = file_obj['Body'].read()
            #return file_content
            # Print the content length and a preview to check if data is being retrieved
            content_length = len(file_content)
            print(f"File {key} downloaded with size: {content_length} bytes")
            #print(f"File content preview (first 100 bytes): {file_content[:100]}")

            # Check for empty content
            if content_length == 0:
                print(f"WARNING: No content found in file: s3://{bucket_name}/{key}")
                return []

            # Handle .txt files as plain text
            if key.endswith('.txt'):
                try:
                    text_content = file_content.decode('utf-8')
                    print(f"Text content loaded successfully for file: {key}")
                    return text_content
                    #return TextLoader(text_content).load()
                except UnicodeDecodeError:
                    # Log and retry decoding with errors ignored
                    text_content = file_content.decode('utf-8', errors='ignore')
                    print(f"WARNING: Non-UTF-8 encoding detected in file {bucket_name}/{key}, decoding with errors ignored.")
                    return TextLoader(text_content).load()

            # Handle PDF files by saving to a temporary file for PyPDFLoader
            elif key.endswith('.pdf'):
                with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as temp_file:
                    temp_file.write(file_content)
                    temp_file_path = temp_file.name

                print(f"Loaded binary file into temporary file: {temp_file_path}")
                loaded_data = PyPDFLoader(temp_file_path).load()

                # Clean up the temporary file
                os.remove(temp_file_path)
                return loaded_data

        except boto3.exceptions.S3UploadFailedError:
            print(f"ERROR: Failed to access file {bucket_name}/{key}: Network issue or permissions error.")
        except Exception as e:
            print(f"ERROR: Attempt {attempt + 1} - Error loading file {bucket_name}/{key}: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying ({attempt + 1}/{max_retries})...")
            else:
                print(f"ERROR: Failed to load file after {max_retries} attempts: s3://{bucket_name}/{key}")
                return []

    # Return empty if all attempts fail
    return []


In [1]:
import boto3
s3 = boto3.client('s3')
key = "verra/program_notices/txt/Verra Extends Validation Deadline for Plastic Projects.txt"
bucket_name = 'ccex-climate-proj'
#a = load_file_from_s3(key, bucket_name, 1)

In [2]:
from langchain_community.document_loaders import S3FileLoader

In [3]:
s3.get_object(Bucket=bucket_name, Key=key)#['Body'].read()

{'ResponseMetadata': {'RequestId': 'J0GDZ4158N099D1P',
  'HostId': 'VoZPqxBocKCdIGkOMAyMHvFZRV5NK4mPtRUUB9QdglPbpM9tsoGD1VatUrEtqKioUmD8mcjFQlY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'VoZPqxBocKCdIGkOMAyMHvFZRV5NK4mPtRUUB9QdglPbpM9tsoGD1VatUrEtqKioUmD8mcjFQlY=',
   'x-amz-request-id': 'J0GDZ4158N099D1P',
   'date': 'Mon, 04 Nov 2024 13:04:07 GMT',
   'last-modified': 'Mon, 04 Nov 2024 07:16:30 GMT',
   'etag': '"46c046c9b56928f41c7674d76051cb4d"',
   'x-amz-server-side-encryption': 'AES256',
   'accept-ranges': 'bytes',
   'content-type': 'binary/octet-stream',
   'content-length': '1022',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2024, 11, 4, 7, 16, 30, tzinfo=tzutc()),
 'ContentLength': 1022,
 'ETag': '"46c046c9b56928f41c7674d76051cb4d"',
 'ContentType': 'binary/octet-stream',
 'ServerSideEncryption': 'AES256',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x10a5f2e80>}

In [14]:
#key = "verra/program_notices/txt/Verra Extends Validation Deadline for Plastic Projects.txt"
key = "verra/methodology/pdf/Improved-Grazing-Management-Methodology-v2-4.pdf"
S3FileLoader(bucket=bucket_name, key=key).load()

[Document(metadata={'source': 's3://ccex-climate-proj/verra/methodology/pdf/Improved-Grazing-Management-Methodology-v2-4.pdf'}, page_content="VCS Methodology for\n\nAgricultural Land Management Improved Grassland Management\n\nMark Dangerfield Charlie Wilson James Schultz Alex Nimz\n\nNovember 2010\n\nIGM Methodology\n\n' GreenCollar Pty Ltd\n\nP a g e | 1\n\nAgricultural Land Management – Improved Grassland Management\n\nVersion 2.4 November 2010\n\nScope\n\nThis methodology is for estimating and monitoring the greenhouse gas emission reductions resulting from improved grassland management projects.\n\nIGM Methodology\n\n' GreenCollar Pty Ltd\n\nP a g e | 2\n\nTable of Contents\n\nScope .................................................................................................................................... 2 Table of Contents ................................................................................................................. 3 PART 1 - SOURCES, DEFINITIONS AND 

In [8]:
import sys
!{sys.executable} -m pip install "unstructured[pdf]"

  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached pdfminer.six-20240706-py3-none-any.whl.metadata (4.1 kB)
  Using cached google_cloud_vision-3.8.0-py2.py3-none-any.whl.metadata (5.3 kB)
  Using cached effdet-0.4.1-py3-none-any.whl.metadata (33 kB)
  Using cached unstructured_inference-0.8.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached unstructured.pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
  Using cached layoutparser-0.3.4-py3-none-any.whl.metadata (7.7 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached timm-1.0.11-py3-none-any.whl.metadata (48 kB)
  Using cached grpcio_status-1.67.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached antlr4-python3-runtime-4.9.3.tar.gz (117 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached iopath-0.1.10.tar.gz (42 kB)
  Installing build dependencies

In [13]:
!pip show unstructured

Name: unstructured
Version: 0.16.4
Summary: A library that prepares raw documents for downstream ML tasks.
Home-page: https://github.com/Unstructured-IO/unstructured
Author: Unstructured Technologies
Author-email: devops@unstructuredai.io
License: Apache-2.0
Location: /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages
Requires: backoff, beautifulsoup4, chardet, dataclasses-json, emoji, filetype, html5lib, langdetect, lxml, nltk, numpy, psutil, python-iso639, python-magic, python-oxmsg, rapidfuzz, requests, tqdm, typing-extensions, unstructured-client, wrapt
Required-by: 


In [7]:
!sudo yum install -y file

Password:
sudo: a password is required


In [8]:
!pip install libmagic

  Using cached libmagic-1.0.tar.gz (3.7 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for libmagic: filename=libmagic-1.0-py3-none-any.whl size=4270 sha256=dc657f9fae19f0ecddd5045f5398c8e643fdec9f00cdebcf8c88c082b9685cc4
  Stored in directory: /Users/khoa/Library/Caches/pip/wheels/3e/35/13/53c696ff5f3523479505ec5ed4156198671fb598fe678178bf
Successfully built libmagic
